In [1]:
!rm -rf dl-projects
!git clone https://github.com/mengwangk/dl-projects
!cd dl-projects && ls 

Cloning into 'dl-projects'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 297 (delta 44), reused 51 (delta 21), pack-reused 219
Receiving objects: 100% (297/297), 8.28 MiB | 1.35 MiB/s, done.
Resolving deltas: 100% (149/149), done.
01_1_tensor.ipynb	    50_RL.ipynb		  lstm_s1.ipynb
01_2_correlation.ipynb	    999_Links.ipynb	  lstm_s2.ipynb
01_3_preprocessing.ipynb    archive		  lstm_s3.ipynb
01_4_numpy.ipynb	    d01_download.ipynb	  lstm_s4.ipynb
02_feature_engineer.ipynb   d02_0_features.ipynb  lstm_s5.ipynb
03_feature_selection.ipynb  d02_1_features.ipynb  lstm_s6.ipynb
04_basic_nn.ipynb	    d03_train_dl.ipynb	  notebook2script.py
05_calculus.ipynb	    d04_train_ml.ipynb	  README.md
06_stats.ipynb		    datasets		  run_notebook.py
07_algebra.ipynb	    exp			  scraper
08_probability.ipynb	    featurelib		  stock.ipynb
09_evaluation.ipynb	    LICENSE


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import scipy as sp

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# DATASET = Path("datasets/lotto/data_processed.csv")
DATASET = Path("dl-projects/datasets/lotto/data_processed.csv")

# The Answer to the Ultimate Question of Life, the Universe, and Everything.
np.random.seed(42)

In [ ]:
# Read into data frame
dataset = pd.read_csv(DATASET, header=0, sep=',', quotechar='"', parse_dates=['DrawDate'], dtype={'PrizeType': str})

In [4]:
dataset.head(10)

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,1st_digit,2nd_digit,3rd_digit,4th_digit,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,1_2,1_3,1_4,1_5,1_6,1_7,1_8,1_9,2_3,2_4,2_5,...,pos_1_0_freq,pos_1_1_freq,pos_1_2_freq,pos_1_3_freq,pos_1_4_freq,pos_1_5_freq,pos_1_6_freq,pos_1_7_freq,pos_1_8_freq,pos_1_9_freq,pos_2_0_freq,pos_2_1_freq,pos_2_2_freq,pos_2_3_freq,pos_2_4_freq,pos_2_5_freq,pos_2_6_freq,pos_2_7_freq,pos_2_8_freq,pos_2_9_freq,pos_3_0_freq,pos_3_1_freq,pos_3_2_freq,pos_3_3_freq,pos_3_4_freq,pos_3_5_freq,pos_3_6_freq,pos_3_7_freq,pos_3_8_freq,pos_3_9_freq,pos_4_0_freq,pos_4_1_freq,pos_4_2_freq,pos_4_3_freq,pos_4_4_freq,pos_4_5_freq,pos_4_6_freq,pos_4_7_freq,pos_4_8_freq,pos_4_9_freq
0,40792,1992-05-06,1stPrizeNo,19,0,28,0,28,0,0,1,9,1992,5,19,6,2,127,0.01,0.01,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,...,4,4,1,2,2,2,2,4,1,1,5,8,5,4,6,3,4,5,2,4,8,8,4,4,3,5,3,7,2,2,5,8,2,2,5,5,5,6,2,6
1,40792,1992-05-06,2ndPrizeNo,1124,0,2436,0,162,1,1,2,4,1992,5,19,6,2,127,0.01,0.01,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,...,4,4,1,2,2,2,2,4,1,1,5,8,5,4,6,3,4,5,2,4,8,8,4,4,3,5,3,7,2,2,5,8,2,2,5,5,5,6,2,6
2,40792,1992-05-06,3rdPrizeNo,592,0,25,0,11,0,5,9,2,1992,5,19,6,2,127,0.01,0.01,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,...,4,4,1,2,2,2,2,4,1,1,5,8,5,4,6,3,4,5,2,4,8,8,4,4,3,5,3,7,2,2,5,8,2,2,5,5,5,6,2,6
3,40792,1992-05-06,ConsolationNo1,5311,0,515,0,204,5,3,1,1,1992,5,19,6,2,127,0.01,0.01,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,...,4,4,1,2,2,2,2,4,1,1,5,8,5,4,6,3,4,5,2,4,8,8,4,4,3,5,3,7,2,2,5,8,2,2,5,5,5,6,2,6
4,40792,1992-05-06,ConsolationNo10,407,0,2114,0,22,0,4,0,7,1992,5,19,6,2,127,0.01,0.01,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,...,4,4,1,2,2,2,2,4,1,1,5,8,5,4,6,3,4,5,2,4,8,8,4,4,3,5,3,7,2,2,5,8,2,2,5,5,5,6,2,6
5,40792,1992-05-06,ConsolationNo2,1949,0,95,0,95,1,9,4,9,1992,5,19,6,2,127,0.01,0.01,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,...,4,4,1,2,2,2,2,4,1,1,5,8,5,4,6,3,4,5,2,4,8,8,4,4,3,5,3,7,2,2,5,8,2,2,5,5,5,6,2,6
6,40792,1992-05-06,ConsolationNo3,1606,0,351,0,35,1,6,0,6,1992,5,19,6,2,127,0.01,0.01,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,...,4,4,1,2,2,2,2,4,1,1,5,8,5,4,6,3,4,5,2,4,8,8,4,4,3,5,3,7,2,2,5,8,2,2,5,5,5,6,2,6
7,40792,1992-05-06,ConsolationNo4,3775,0,494,0,126,3,7,7,5,1992,5,19,6,2,127,0.01,0.01,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,...,4,4,1,2,2,2,2,4,1,1,5,8,5,4,6,3,4,5,2,4,8,8,4,4,3,5,3,7,2,2,5,8,2,2,5,5,5,6,2,6
8,40792,1992-05-06,ConsolationNo5,6226,0,494,0,232,6,2,2,6,1992,5,19,6,2,127,0.01,0.01,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,...,4,4,1,2,2,2,2,4,1,1,5,8,5,4,6,3,4,5,2,4,8,8,4,4,3,5,3,7,2,2,5,8,2,2,5,5,5,6,2,6
9,40792,1992-05-06,ConsolationNo6,1271,0,410,0,119,1,2,7,1,1992,5,19,6,2,127,0.01,0.01,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,...,4,4,1,2,2,2,2,4,1,1,5,8,5,4,6,3,4,5,2,4,8,8,4,4,3,5,3,7,2,2,5,8,2,2,5,5,5,6,2,6


In [5]:
df = dataset.copy()
len(df.columns)

115

In [6]:
for ind, column in enumerate(df.columns):
    print('"{}"'.format(column), end=', ')
    if ind % 5 == 0:
        print()

"DrawNo", 
"DrawDate", "PrizeType", "LuckyNo", "DaysSinceLastExactMatch", "DaysUntilNextExactMatch", 
"DaysSinceLastAnyMatch", "DaysUntilNextAnyMatch", "1st_digit", "2nd_digit", "3rd_digit", 
"4th_digit", "DrawYear", "DrawMonth", "DrawWeek", "DrawDay", 
"DrawDayofweek", "DrawDayofyear", "CumProbaExactMatch", "CumProbaAnyMatch", "0_1", 
"0_2", "0_3", "0_4", "0_5", "0_6", 
"0_7", "0_8", "0_9", "1_2", "1_3", 
"1_4", "1_5", "1_6", "1_7", "1_8", 
"1_9", "2_3", "2_4", "2_5", "2_6", 
"2_7", "2_8", "2_9", "3_4", "3_5", 
"3_6", "3_7", "3_8", "3_9", "4_5", 
"4_6", "4_7", "4_8", "4_9", "5_6", 
"5_7", "5_8", "5_9", "6_7", "6_8", 
"6_9", "7_8", "7_9", "8_9", "0_0", 
"1_1", "2_2", "3_3", "4_4", "5_5", 
"6_6", "7_7", "8_8", "9_9", "pos_1_0_freq", 
"pos_1_1_freq", "pos_1_2_freq", "pos_1_3_freq", "pos_1_4_freq", "pos_1_5_freq", 
"pos_1_6_freq", "pos_1_7_freq", "pos_1_8_freq", "pos_1_9_freq", "pos_2_0_freq", 
"pos_2_1_freq", "pos_2_2_freq", "pos_2_3_freq", "pos_2_4_freq", "pos_2_5_freq", 
"pos_2_6_freq"

In [7]:
numeric_features = ["DrawNo",
                    "DaysSinceLastExactMatch", "DaysUntilNextExactMatch", 
                    "DaysSinceLastAnyMatch", "DaysUntilNextAnyMatch",
                    "DrawYear", "DrawMonth", "DrawWeek", "DrawDay", "DrawDayofweek", "DrawDayofyear",
                    "CumProbaExactMatch", "CumProbaAnyMatch",
                    "0_1", "0_2", "0_3", "0_4", "0_5", 
                    "0_6", "0_7", "0_8", "0_9", "1_2", "1_3", "1_4", "1_5", 
                    "1_6", "1_7", "1_8", "1_9", "2_3", "2_4", "2_5", "2_6", 
                    "2_7", "2_8", "2_9", "3_4", "3_5", "3_6", "3_7", "3_8", 
                    "3_9", "4_5", "4_6", "4_7", "4_8", "4_9", "5_6", "5_7", 
                    "5_8", "5_9", "6_7", "6_8", "6_9", "7_8", "7_9", "8_9", 
                    "0_0", "1_1", "2_2", "3_3", "4_4", "5_5", "6_6", "7_7", 
                    "8_8", "9_9",
                    "pos_1_0_freq", "pos_1_1_freq", "pos_1_2_freq", "pos_1_3_freq", "pos_1_4_freq", "pos_1_5_freq", 
                    "pos_1_6_freq", "pos_1_7_freq", "pos_1_8_freq", "pos_1_9_freq", "pos_2_0_freq", "pos_2_1_freq", "pos_2_2_freq", "pos_2_3_freq", 
                    "pos_2_4_freq", "pos_2_5_freq", "pos_2_6_freq", "pos_2_7_freq", "pos_2_8_freq", "pos_2_9_freq", "pos_3_0_freq", "pos_3_1_freq", 
                    "pos_3_2_freq", "pos_3_3_freq", "pos_3_4_freq", "pos_3_5_freq", "pos_3_6_freq", "pos_3_7_freq", "pos_3_8_freq", "pos_3_9_freq", 
                    "pos_4_0_freq", "pos_4_1_freq", "pos_4_2_freq", "pos_4_3_freq", "pos_4_4_freq", "pos_4_5_freq", "pos_4_6_freq", "pos_4_7_freq", 
                    "pos_4_8_freq", "pos_4_9_freq"
                   ]
categorical_features = ["PrizeType"]
datetime_features = ["DrawDate"]
target_feature = ["LuckyNo"]
target_features_drilled = ["1st_digit", "2nd_digit", "3rd_digit", "4th_digit"]

# mean_features = ["TotalMean", "1stDigitMean", "2ndDigitMean", "3rdDigitMean", "4thDigitMean"]

len(numeric_features)

108

### Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
nf_scaled = scaler.fit_transform(df[numeric_features])
print(scaler.mean_)
print(scaler.inverse_transform(nf_scaled))

[2.68299492e+05 7.64409447e+02 7.64409447e+02 6.61790529e+01
 6.61790529e+01 2.00575478e+03 6.50164799e+00 2.64524280e+01
 1.56282136e+01 4.05405405e+00 1.82380356e+02 6.23821807e-02
 9.30387206e-01 5.47741134e+03 5.42231773e+03 5.55793957e+03
 5.45725005e+03 5.41884179e+03 5.39399253e+03 5.45006834e+03
 5.40091365e+03 5.48733092e+03 5.59624874e+03 5.59605845e+03
 5.66231993e+03 5.50389519e+03 5.52639903e+03 5.57656537e+03
 5.56644847e+03 5.53951791e+03 5.56605823e+03 5.50430521e+03
 5.41334476e+03 5.44459635e+03 5.38095957e+03 5.42613052e+03
 5.39721402e+03 5.51594419e+03 5.50239354e+03 5.51964821e+03
 5.52140628e+03 5.57041903e+03 5.55934102e+03 5.39769875e+03
 5.39214217e+03 5.38072270e+03 5.50358471e+03 5.51762360e+03
 5.36897956e+03 5.34769128e+03 5.42560338e+03 5.47009383e+03
 5.51257108e+03 5.45700198e+03 5.38235377e+03 5.54290200e+03
 5.28726368e+03 5.49614568e+03 2.71367765e+03 2.79092156e+03
 2.75304614e+03 2.85032806e+03 2.73804461e+03 2.66042936e+03
 2.66422610e+03 2.750623

In [9]:
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MultiLabelBinarizer
from sklearn.feature_extraction import DictVectorizer, FeatureHasher

onehot_encoder = OneHotEncoder()
cf_scaled = onehot_encoder.fit_transform(df[categorical_features])
onehot_encoder.categories_, cf_scaled.shape, len(cf_scaled.toarray()), cf_scaled.toarray()

([array(['1stPrizeNo', '2ndPrizeNo', '3rdPrizeNo', 'ConsolationNo1',
         'ConsolationNo10', 'ConsolationNo2', 'ConsolationNo3',
         'ConsolationNo4', 'ConsolationNo5', 'ConsolationNo6',
         'ConsolationNo7', 'ConsolationNo8', 'ConsolationNo9', 'SpecialNo1',
         'SpecialNo10', 'SpecialNo2', 'SpecialNo3', 'SpecialNo4',
         'SpecialNo5', 'SpecialNo6', 'SpecialNo7', 'SpecialNo8',
         'SpecialNo9'], dtype=object)],
 (104673, 23),
 104673,
 array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]))

In [10]:
# Features
nf_scaled.shape, cf_scaled.shape

((104673, 108), (104673, 23))

In [11]:
# Join the 2 arrays
features = sp.sparse.hstack((nf_scaled, cf_scaled))
features.shape

(104673, 131)

In [12]:
labels = scaler.fit_transform(df[target_feature])
labels, scaler.inverse_transform(labels)

(array([[-1.71731986],
        [-1.33434554],
        [-1.51872774],
        ...,
        [ 0.0263397 ],
        [ 1.42064349],
        [-0.69247364]]), array([[  19.],
        [1124.],
        [ 592.],
        ...,
        [5050.],
        [9073.],
        [2976.]]))

### Correlation

In [13]:
features.shape, labels.shape

((104673, 131), (104673, 1))

In [14]:
features.dtype, labels.dtype

(dtype('float64'), dtype('float64'))

In [ ]:
for ind, f in enumerate(nf_scaled[0]):
    print(ind, numeric_features[ind], '\t\t', np.corrcoef(nf_scaled[:, ind], labels[:, 0])[0][1])
    # print(nf_scaled[:, ind].shape, labels[:, 0].shape)
    # print()

#### Drop features

In [ ]:
# drop DrawDay
numeric_features = ["DrawNo",
                    "DaysSinceLastExactMatch", "DaysUntilNextExactMatch", 
                    "DaysSinceLastAnyMatch", "DaysUntilNextAnyMatch",
                    "DrawYear", "DrawMonth", "DrawWeek", "DrawDayofweek", "DrawDayofyear",
                    "CumProbaExactMatch", "CumProbaAnyMatch"
                   ]
categorical_features = ["PrizeType"]
datetime_features = ["DrawDate"]
target_feature = ["LuckyNo"]

In [17]:
nf_scaled = scaler.fit_transform(df[numeric_features])
for ind, f in enumerate(nf_scaled[0]):
    print(numeric_features[ind], '\t\t', np.corrcoef(nf_scaled[:, ind], labels[:, 0])[0][1])

DrawNo 		 -0.0032182204064142147
DaysSinceLastExactMatch 		 0.005984842985229681
DaysUntilNextExactMatch 		 0.005984842985229679
DaysSinceLastAnyMatch 		 0.002328232886996674
DaysUntilNextAnyMatch 		 0.0033065825999482666
DrawYear 		 -0.003070982129500965
DrawMonth 		 -0.0029849057887388272
DrawWeek 		 -0.0027621409428476604
DrawDayofweek 		 0.0043796593856403245
DrawDayofyear 		 -0.00289589777455866
CumProbaExactMatch 		 -0.009410044867088282
CumProbaAnyMatch 		 -0.003994047653223088


In [18]:
# Validation
corr_matrix = df[numeric_features + categorical_features + target_feature].corr()
print(corr_matrix['LuckyNo'].sort_values(ascending=False))

LuckyNo                    1.000000
DaysUntilNextExactMatch    0.005985
DaysSinceLastExactMatch    0.005985
DrawDayofweek              0.004380
DaysUntilNextAnyMatch      0.003307
DaysSinceLastAnyMatch      0.002328
DrawWeek                  -0.002762
DrawDayofyear             -0.002896
DrawMonth                 -0.002985
DrawYear                  -0.003071
DrawNo                    -0.003218
CumProbaAnyMatch          -0.003994
CumProbaExactMatch        -0.009410
Name: LuckyNo, dtype: float64


In [19]:
df = dataset[target_feature + target_features_drilled + categorical_features + numeric_features + datetime_features].copy()
df.columns

Index(['LuckyNo', '1st_digit', '2nd_digit', '3rd_digit', '4th_digit',
       'PrizeType', 'DrawNo', 'DaysSinceLastExactMatch',
       'DaysUntilNextExactMatch', 'DaysSinceLastAnyMatch',
       'DaysUntilNextAnyMatch', 'DrawYear', 'DrawMonth', 'DrawWeek',
       'DrawDayofweek', 'DrawDayofyear', 'CumProbaExactMatch',
       'CumProbaAnyMatch', 'DrawDate'],
      dtype='object')

In [20]:
df.describe() # all data

,LuckyNo,1st_digit,2nd_digit,3rd_digit,4th_digit,DrawNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,DrawYear,DrawMonth,DrawWeek,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch
count,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000
mean,4974.001777,4.475471,4.490986,4.495935,4.472577,268299.492419,764.409447,764.409447,66.179053,66.179053,2005.754779,6.501648,26.452428,4.054054,182.380356,0.062382,0.930387
std,2885.324727,2.871106,2.875512,2.876148,2.872390,131388.886981,832.538166,832.538166,93.325465,93.325465,7.809743,3.462904,15.123744,1.843042,105.868193,0.038001,0.675621
min,0.000000,0.000000,0.000000,0.000000,0.000000,40792.000000,0.000000,0.000000,0.000000,0.000000,1992.000000,1.000000,1.000000,1.000000,1.000000,0.010000,0.010000
25%,2463.000000,2.000000,2.000000,2.000000,2.000000,154499.000000,157.000000,157.000000,15.000000,15.000000,1999.000000,3.000000,13.000000,2.000000,90.000000,0.030000,0.380000
50%,4952.000000,4.000000,4.000000,4.000000,4.000000,268306.000000,504.000000,504.000000,39.000000,39.000000,2006.000000,7.000000,26.000000,5.000000,182.000000,0.060000,0.800000
75%,7463.000000,7.000000,7.000000,7.000000,7.000000,382113.000000,1095.000000,1095.000000,83.000000,83.000000,2013.000000,10.000000,40.000000,6.000000,274.000000,0.090000,1.310000
max,9999.000000,9.000000,9.000000,9.000000,9.000000,495819.000000,8087.000000,8087.000000,4113.000000,4113.000000,2019.000000,12.000000,53.000000,6.000000,366.000000,0.270000,2.930000


In [21]:
# 1/3 of data
df.head( int(len(df) / 3)).describe()

,LuckyNo,1st_digit,2nd_digit,3rd_digit,4th_digit,DrawNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,DrawYear,DrawMonth,DrawWeek,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch
count,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000,34891.000000
mean,4981.949385,4.481041,4.512740,4.515663,4.477831,116577.862887,474.335846,985.147431,67.194778,71.686452,1996.649967,6.611734,26.945946,4.127884,185.798945,0.027447,0.317357
std,2880.783177,2.866376,2.874339,2.867806,2.871029,43769.968055,582.554770,968.142259,89.553446,102.626954,2.777329,3.391292,14.819038,1.746514,103.679451,0.016649,0.231094
min,0.000000,0.000000,0.000000,0.000000,0.000000,40792.000000,0.000000,0.000000,0.000000,0.000000,1992.000000,1.000000,1.000000,1.000000,1.000000,0.010000,0.010000
25%,2490.000000,2.000000,2.000000,2.000000,2.000000,78694.000000,0.000000,283.000000,15.000000,17.000000,1994.000000,4.000000,14.000000,2.000000,98.000000,0.010000,0.130000
50%,4952.000000,4.000000,5.000000,5.000000,4.000000,116597.000000,258.000000,689.000000,39.000000,42.000000,1997.000000,7.000000,27.000000,5.000000,187.000000,0.020000,0.270000
75%,7471.000000,7.000000,7.000000,7.000000,7.000000,154499.000000,731.000000,1375.000000,85.000000,90.000000,1999.000000,10.000000,40.000000,6.000000,274.000000,0.040000,0.450000
max,9999.000000,9.000000,9.000000,9.000000,9.000000,192401.000000,3370.000000,8087.000000,2435.000000,4113.000000,2001.000000,12.000000,53.000000,6.000000,366.000000,0.140000,1.070000


In [22]:
# 1/2 of data
df.head( int(len(df) / 2)).describe()

,LuckyNo,1st_digit,2nd_digit,3rd_digit,4th_digit,DrawNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,DrawYear,DrawMonth,DrawWeek,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch
count,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000,52336.000000
mean,4974.957505,4.475180,4.502064,4.509439,4.477148,154510.398674,616.939812,962.510738,67.461843,70.345269,1999.004987,6.491765,26.424775,4.124178,182.047921,0.036196,0.470292
std,2885.343416,2.871423,2.877207,2.872626,2.874466,65688.055585,714.748102,939.894763,91.973476,100.754145,4.103606,3.445370,15.065965,1.776259,105.388257,0.022192,0.341735
min,0.000000,0.000000,0.000000,0.000000,0.000000,40792.000000,0.000000,0.000000,0.000000,0.000000,1992.000000,1.000000,1.000000,1.000000,1.000000,0.010000,0.010000
25%,2470.000000,2.000000,2.000000,2.000000,2.000000,97595.000000,59.000000,277.000000,15.000000,17.000000,1995.000000,4.000000,13.000000,2.000000,91.000000,0.020000,0.190000
50%,4941.500000,4.000000,4.000000,4.000000,4.000000,154499.000000,378.000000,675.000000,39.000000,42.000000,1999.000000,6.000000,26.000000,5.000000,181.000000,0.030000,0.400000
75%,7465.000000,7.000000,7.000000,7.000000,7.000000,211403.000000,913.000000,1348.000000,84.000000,87.000000,2003.000000,9.000000,39.000000,6.000000,273.000000,0.050000,0.670000
max,9999.000000,9.000000,9.000000,9.000000,9.000000,268306.000000,5124.000000,8087.000000,3940.000000,4113.000000,2006.000000,12.000000,53.000000,6.000000,366.000000,0.170000,1.530000


In [23]:
df['LuckyNo'].mean(), df['1st_digit'].mean(), df['2nd_digit'].mean(), df['3rd_digit'].mean(), df['4th_digit'].mean()

(4974.00177696254,
 4.4754712294478995,
 4.490986214209968,
 4.495934959349594,
 4.47257650014808)

In [24]:
# Derive the means

df["TotalMean"] = 0
df["1stDigitMean"] = 0
df["2ndDigitMean"] = 0 
df["3rdDigitMean"] = 0 
df["4thDigitMean"] = 0

for index, draw_date in enumerate(df.DrawDate.unique()):
    if (index + 1) % 1000 == 0:
        print(f'Processing {index+1}')
    df.loc[df.DrawDate == draw_date, 'TotalMean'] = df[df.DrawDate <= draw_date]['LuckyNo'].mean()
    df.loc[df.DrawDate == draw_date, '1stDigitMean'] = df[df.DrawDate <= draw_date]['1st_digit'].mean()
    df.loc[df.DrawDate == draw_date, '2ndDigitMean'] = df[df.DrawDate <= draw_date]['2nd_digit'].mean()
    df.loc[df.DrawDate == draw_date, '3rdDigitMean'] = df[df.DrawDate <= draw_date]['3rd_digit'].mean()
    df.loc[df.DrawDate == draw_date, '4thDigitMean'] = df[df.DrawDate <= draw_date]['4th_digit'].mean()
    # print(len(df[df.DrawDate <= draw_date]['LuckyNo']))
    # print(draw_date, df[df.DrawDate == draw_date]['LuckyNo'].mean())
    # if (index + 1 % 100 == 0):
        #break
    #print()

Processing 1000
Processing 2000
Processing 3000
Processing 4000


In [26]:
df.tail(26)

,LuckyNo,1st_digit,2nd_digit,3rd_digit,4th_digit,PrizeType,DrawNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,DrawYear,DrawMonth,DrawWeek,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch,DrawDate,TotalMean,1stDigitMean,2ndDigitMean,3rdDigitMean,4thDigitMean
104647,1144,1,1,4,4,SpecialNo7,495719,1386,0,21,0,2019,6,23,5,159,0.11,0.64,2019-06-08,4974.168982,4.475633,4.491037,4.495968,4.472566
104648,7351,7,3,5,1,SpecialNo8,495719,962,0,27,0,2019,6,23,5,159,0.16,2.61,2019-06-08,4974.168982,4.475633,4.491037,4.495968,4.472566
104649,6775,6,7,7,5,SpecialNo9,495719,76,0,76,0,2019,6,23,5,159,0.11,1.16,2019-06-08,4974.168982,4.475633,4.491037,4.495968,4.472566
104650,397,0,3,9,7,1stPrizeNo,495819,406,0,1,0,2019,6,23,6,160,0.11,2.63,2019-06-09,4974.001777,4.475471,4.490986,4.495935,4.472577
104651,1232,1,2,3,2,2ndPrizeNo,495819,945,0,36,0,2019,6,23,6,160,0.12,1.34,2019-06-09,4974.001777,4.475471,4.490986,4.495935,4.472577
104652,1219,1,2,1,9,3rdPrizeNo,495819,197,0,109,0,2019,6,23,6,160,0.12,1.22,2019-06-09,4974.001777,4.475471,4.490986,4.495935,4.472577
104653,8611,8,6,1,1,ConsolationNo1,495819,2948,0,15,0,2019,6,23,6,160,0.05,1.24,2019-06-09,4974.001777,4.475471,4.490986,4.495935,4.472577
104654,5031,5,0,3,1,ConsolationNo10,495819,564,0,50,0,2019,6,23,6,160,0.09,2.77,2019-06-09,4974.001777,4.475471,4.490986,4.495935,4.472577
104655,4622,4,6,2,2,ConsolationNo2,495819,176,0,57,0,2019,6,23,6,160,0.12,1.29,2019-06-09,4974.001777,4.475471,4.490986,4.495935,4.472577
104656,483,0,4,8,3,ConsolationNo3,495819,1183,0,57,0,2019,6,23,6,160,0.09,2.50,2019-06-09,4974.001777,4.475471,4.490986,4.495935,4.472577


In [ ]:
df.to_csv("dl-projects/datasets/lotto/data_processed_2.csv", index=False, header=True)

In [30]:
!cd dl-projects && git config --global user.email "mengwangk@gmail.com" && git config --global user.name "mengwangk" && git add -A && git commit -m 'Updated files' && git push

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
